# Funções úteis


In [1]:
# @title Derivadas Numéricas
import casadi as ca


def dydz(y: ca.MX, dz: float) -> ca.MX:
    """
    Computes the first derivative of y with respect to z.
    """
    dydz = ca.MX.zeros(y.shape[0])

    dydz[0] = (-3 * y[0] + 4 * y[1] - y[2]) / (2 * dz)  # Forward at inlet
    dydz[1:-1] = (y[2:] - y[:-2]) / (2 * dz)  # Central
    dydz[-1] = (3 * y[-1] - 4 * y[-2] + y[-3]) / (2 * dz)  # Backward at outlet
    return dydz


def d2ydz2(y: ca.MX, dz: float) -> ca.MX:
    d2ydz2 = ca.MX.zeros(y.shape[0])

    d2ydz2[0] = (2 * y[0] - 5 * y[1] + 4 * y[2] - y[3]) / dz**2  # Forward at inlet
    d2ydz2[1:-1] = (y[2:] - 2 * y[1:-1] + y[:-2]) / dz**2  # Central
    d2ydz2[-1] = (
        2 * y[-1] - 5 * y[-2] + 4 * y[-3] - y[-4]
    ) / dz**2  # Backward at outlet
    return d2ydz2


# Simulação


In [2]:
# @title Constantes
from typing import Final

ng: Final[int] = 3

Rg: Final = 8.314
Rp: Final = 1e-3
roap: Final = 549
Cps: Final = 1200
epsb: Final = 0.352
rob: Final = (1 - epsb) * roap
apm: Final = 3 / Rp
dp: Final = 2 * Rp
L: Final = 0.323
Dw: Final = 0.0211
row: Final = 8238
Cpw: Final = 500
alfaw: Final = 457
alfawl: Final = 499
Qfeed_SLPM: Final = 0.43
bed_area: Final = 3.1415 * (Dw / 2) ** 2
nocycles: Final = 20
betapeq: Final = 0.04
betadepco: Final = 0.07
betablow: Final = 0.018
dax: Final = 6.9e-5
kf: Final = 2.0e-2
λ: Final = 0.6
hf: Final = 128
visc: Final = 1.7e-5
U: Final = 37
hw: Final = 50
Cpmix: Final = 31
Cvmix: Final = 23
Mwi: Final = ca.MX([4.400e-02, 2.8e-02, 2e-3])
dpi: Final = ca.MX([7.7e-7, 9.8e-7, 2.5e-6])
MDH1: Final = ca.MX([21885, 13351, 35260])
kinf: Final = ca.MX([5.78e-5, 3.36e-4, 9.39e-7])
qsat: Final = ca.MX([8.509, 3.482, 0.294])
Cvi: Final = ca.MX([29.8, 20.6, 12.5])
eta: Final = 0.85
# gama: Final = cpmix / cvmix

yin_feed: Final = ca.MX([0.3, 0.22, 0.48])
yin_peq: Final = ca.MX([0, 1, 0])
yin_press: Final = ca.MX([0, 0, 1])
yin_purge: Final = ca.MX([0, 0.32, 0.68])
yin_rinse: Final = ca.MX([1, 0, 0])
yin_rr: Final = ca.MX([1, 0, 0])
Phigh: Final = 4.8e5
Plow: Final = 0.55e5
Pblow: Final = 1e5
Prinse: Final = 1e5
Ppeqd: Final = 33e5
Ppeqp: Final = 11e5
Ppurge: Final = 1e5
Ppress: Final = 4e5
Pdepco: Final = 1e5
Performance: Final = 100
Pinter1: Final = 1.5e5
Pinter2: Final = 11e5
qrr: Final = 1e-5
qeq: Final = 0.3
countcycle: Final = 1
Qn_r: Final = 0.57
Qn_p: Final = 0.305
Tfeed: Final = 679.462
Tpress: Final = 0
Teq: Final = 0
Tpurge: Final = 80
Trinse: Final = 187
Tdepco: Final = 0
Tinlet: Final = 305.059

u0inlet: Final = (Qfeed_SLPM * 100000 / Phigh * Tinlet / 273.15 / 60 / 1000) / bed_area
Qfeed: Final = u0inlet * bed_area
Cinlet: Final = Phigh / Rg / Tinlet
Tinf: Final = Tinlet


In [3]:
# @title Configurações
import numpy as np

n_z = 20  # Number of discrete points in space
z = np.linspace(0, L, n_z)  # Spatial grid
dz = z[1] - z[0]  # Spatial step size

n_t = 1000  # Number of discrete points in time
tf = Tfeed  # Final simulation time
t = np.linspace(0, tf, n_t)
dt = t[1] - t[0]


In [4]:
# @title Outras Funções


def partial(y):
    return dydz(y, dz)


def partial2(y):
    return d2ydz2(y, dz)


In [5]:
# @title Definição do Modelo

n_x_vars = (
    (n_z - 2) * ng  # Cg_CO2, Cg_CO, Cg_H2
    + (n_z) * ng  # q_CO2, q_CO, q_H2
    + (n_z - 2)  # Tg
    + n_z  # Tp
    + n_z  # Tw
)

x_z_vars = (
    6  # Cg_CO2(0), Cg_CO2(L), Cg_CO(0), Cg_CO(L), Cg_H2(0), Cg_H2(L)
    + 2  # Tg(0), Tg(L)
    + n_z * ng  # Csi_CO2, Csi_CO, Csi
    + 2  # P
    + n_z  # u
)


# --- Output variables ---
# Diferential Variables
x = ca.MX.sym("x", n_x_vars)  # type: ignore
ix = 0
# Algebric Variables
z = ca.MX.sym("z", x_z_vars)  # type: ignore
iz = 0

# Molar concentration in the gas phase of components
Cg_CO2 = ca.vertcat(z[iz + 0], x[0 + ix : n_z - 2 + ix], z[iz + 1])
Cg_CO = ca.vertcat(z[iz + 2], x[n_z - 2 + ix : 2 * (n_z - 2) + ix], z[iz + 3])
Cg_H2 = ca.vertcat(z[iz + 4], x[2 * (n_z - 2) + ix : 3 * (n_z - 2) + ix], z[iz + 5])
iz += 2 * ng
ix += 3 * (n_z - 2)
Cg = ca.vertcat(Cg_CO2.T, Cg_CO.T, Cg_H2.T)

# Particle adsorbed concentration of components
q = (x[ix : ix + n_z * 3]).reshape((ng, n_z))
ix += n_z * 3

# Temperatures
Tg = ca.vertcat(z[iz + 0], x[0 + ix : n_z - 2 + ix], z[iz + 1])
iz += 2
ix += n_z - 2

Tp = x[ix : ix + n_z]
ix += n_z
Tw = x[ix : ix + n_z]
ix += n_z

# --- Algebraic Variables ---
Cs = (z[iz : iz + n_z * ng]).reshape((ng, n_z))
iz += n_z * ng

u0 = z[iz : iz + n_z]
iz += n_z

P_z = z[iz : iz + 2]  # Only boundary values must be solved by the casadi
iz += 2
P_MX = ca.MX.zeros(n_z - 2)  # type: ignore # The rest can be solved explicitly
P = ca.vertcat(P_z[0], P_MX, P_z[1])

# --- Algebraic Equations ---
# Explicit
Cgt = ca.sum1(Cg).T

y = ca.MX.zeros((ng, n_z))  # type: ignore
for i in range(ng):
    y[i, :] = Cg[i, :] / Cgt.T

Mw = ca.mtimes(Mwi.T, y).T

keq = ca.MX.zeros((ng, n_z))  # type: ignore
for i in range(ng):
    keq[i, :] = kinf[i] * ca.exp(MDH1[i] / (Rg * Tp.T))

Pmpbar = ca.MX.zeros((ng, n_z))  # type: ignore
for i in range(ng):
    Pmpbar[i, :] = Cs[i, :] * Rg * Tp.T / 1e5

q_star = ca.MX.zeros((ng, n_z))  # type: ignore
for i in range(ng):
    q_star[i, :] = (
        qsat[i] * (keq[i] * Pmpbar[i]) / (1 + (ca.sum1(ca.fabs(keq * Pmpbar))))
    )

P[1:-1] = (Cgt * Tg * Rg)[1:-1]

# Residuals (Implicit)
# Gambiarra para saber se todos os res foram inicializados. Basta checar pelo valor 9999999 no final.
res = ca.MX.ones(z.shape[0]) * 9999999
offset = 0

res[offset : offset + n_z] = -partial(P) - (
    150 * visc * (1 - epsb) ** 2 / (epsb**3 * dp**2) * u0
    + 1.75 * (1 - epsb) / (epsb**3 * dp) * Cgt * Mw * u0 * ca.fabs(u0)
)
offset += n_z

for i in range(ng):
    res[offset : offset + n_z] = 15 * dpi[i] / P * Phigh / (Rp**2) * (
        q_star[i] - q[i]
    ) - (apm * kf * (Cg[i] - Cs[i]) / roap)
    offset += n_z

# Boundary Conditions
for i in range(ng):
    dCgidz = partial(Cg[i, :].T)

    res[offset] = Cg[i, 0] - epsb * dax / u0inlet * dCgidz[0] - yin_feed[i] * Cinlet
    offset += 1
    res[offset] = dCgidz[-1]
    offset += 1

cond = P[-1] < Phigh
res[offset] = ca.if_else(
    cond,
    0.5 * u0inlet * Cinlet - u0[0] * Cgt[0],
    u0inlet * Cinlet - u0[0] * Cgt[0],
)
offset += 1
res[offset] = ca.if_else(
    cond,
    u0[-1],
    Phigh - P[-1],
)
offset += 1

dTgdz = partial(Tg)
res[offset] = Tg[0] - λ / u0inlet / Cinlet / Cpmix * dTgdz[0] - Tinlet
offset += 1
res[offset] = dTgdz[-1]
offset += 1

print(
    "O n° de variaveis algébricas deve ser igual ao n° de equações algébricas (residuos):"
)
print(z.shape)
print(offset)

# --- Differential Equations ---
dCgdt = ca.MX.zeros((ng, n_z - 2))  # type: ignore
for i in range(ng):
    dCgdt[i, :] = (
        (
            (
                epsb
                * dax
                * (partial(Cgt) * partial(y[i, :].T) + Cgt * partial2(y[i, :].T))
            )
            - partial(u0 * Cg[i, :].T)
            - (1 - epsb) * apm * kf * (Cg[i] - Cs[i])
        )
        / epsb
    )[1:-1]

dqdt = ca.MX.zeros((ng, n_z))  # type: ignore
for i in range(ng):
    dqdt[i, :] = 15 * dpi[i] / P.T * Phigh / Rp**2 * (q_star[i, :] - q[i, :])

dTgdt = (
    (λ * partial2(Tg)[1:-1])
    - u0[1:-1] * Cgt[1:-1] * Cpmix * dTgdz[1:-1]
    + epsb * Rg * Tg[1:-1] * ca.sum1(dCgdt).T
    - (1 - epsb) * apm * hf * (Tg[1:-1] - Tp[1:-1])
    - 4 * hw / Dw * (Tg[1:-1] - Tw[1:-1])
) / (epsb * Cgt[1:-1] * Cvmix)

dTpdt = (rob * (ca.sum1(MDH1 * dqdt).T) + (1 - epsb) * apm * hf * (Tg - Tp)) / (
    (1 - epsb) * (roap * (ca.sum1(q * Cvi).T) + roap * Cps)
)

dTwdt = (alfaw * hw * (Tg - Tw) - alfawl * U * (Tw - Tinf)) / (row * Cpw)

dxdt = ca.vertcat(ca.vec(dCgdt), ca.vec(dqdt), dTgdt, dTpdt, dTwdt)

print("O n° de variaveis deve ser igual ao n° de equações diferenciais:")
print(x.shape)
print(dxdt.shape)

ode = {"x": x, "z": z, "ode": dxdt, "alg": res}


O n° de variaveis algébricas deve ser igual ao n° de equações algébricas (residuos):
(90, 1)
90
O n° de variaveis deve ser igual ao n° de equações diferenciais:
(172, 1)
(172, 1)


In [6]:
# @title Condições Iniciais

# Pelo gPROMS temos as informações dos y Cgt iniciais.
# Mas y e Cgt são algébricos e não precisam de condição inicial.
# Então calculamos Cg apartir deles:
y0 = np.zeros((ng, n_z - 2))
y0[2, :] = 1
Cgt0 = Phigh / Rg / Tinlet
Cg0 = y0 * Cgt0

# O resto tem no código do gPROMs, sem segredo
q0 = np.zeros((ng, n_z))

Tg0 = np.full(n_z - 2, Tinlet)
Tp0 = np.full(n_z, Tinlet)
Tw0 = np.full(n_z, Tinlet)

x0 = np.concat((Cg0.flatten(), q0.flatten(), Tg0, Tp0, Tw0))
print(x0.shape)


(172,)


In [7]:
solver = ca.integrator("solver", "idas", ode, 0, dt)
sol = solver(x0=x0)

print("Parabens! Vocé conseguiu rodar o integrador do CasADi.")
print("Estamos a:", dt, "segundos sem dar erro. Nosso recorde é de 0 segundos.")


RuntimeError: .../casadi/core/function_internal.cpp:151: Error calling IdasInterface::init for 'solver':
.../casadi/core/integrator.cpp:850: Assertion "!sp_jac_dae_.is_singular()" failed:
Jacobian of the forward problem is structurally rank-deficient. sprank(J)=223<262